In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.mlab as mlab
%matplotlib inline

### 1.Read the dataset (tab, csv, xls, txt, inbuilt dataset)

In [ ]:

df = pd.read_csv("../input/indian-liver-patient-records/indian_liver_patient.csv")
df.head()


### 2.Summarize important observations from the data set 

#### Some pointers which would help you, but don’t be limited by these
#### a.	Find out number of rows; no. & types of variables (continuous, categorical etc.)


In [ ]:
# dataframe dimensions
print("This dataframe has {} rows and {} columns".format(*df.shape))

In [ ]:
df.info()

In [ ]:
numerical_cols = df.select_dtypes(include="number").columns
print('There are',len(numerical_cols),'numeric columns')

In [ ]:
categorical_cols = df.select_dtypes(include="object").columns

print('There are',len(categorical_cols),'categorical columns')

In [ ]:
df.rename(columns={'Dataset':'target'},inplace=True)
df.head()

In [ ]:
target_counts=df['target'].value_counts().values
gender_counts=df['Gender'].value_counts().values

fig1, axes=plt.subplots(nrows=1, ncols=2,figsize=(10,5))


target_sizes=df.groupby('target').size()
axes[0].pie(
    x=target_counts,
    labels=['patient({})'.format(target_sizes[1]),'not patient({})'.format(target_sizes[2])],
    autopct='%1.1f%%'
)
axes[0].set_title("Percentage of liver patient")

gender_sizes=df.groupby('Gender').size()
axes[1].pie(
    x=gender_counts, 
    labels=['male({})'.format(gender_sizes['Male']), 'female({})'.format(gender_sizes['Female'])], 
    autopct="%1.1f%%"
)
axes[1].set_title("Age wise liver patient")

In [ ]:
df['Gender'].value_counts().plot.bar(color='peachpuff')

No.of male is more than female.

#### b.	Calculate five-point summary for numerical variables

In [ ]:
df.describe().T

#### c.	Summarize observations for categorical variables – no. of categories, % observations in each category

In [ ]:
df.dtypes.value_counts()

In [ ]:
cat_col = ['Gender','target']
print('Categorical columns in the dataset are :',cat_col)
print('No of values in each categories   ')
for i in cat_col:
   print('Column = ',i)
   print(df[i].value_counts())

In [ ]:
print('Percentage of categories in each variable')
for i in cat_col:
   print('Column = ',i)
   print(df[i].value_counts()/len(df[i]))    

### 3.Check for defects in the data. Perform necessary actions to ‘fix’ these defects (5 Marks)

#### Some pointers which would help you, but don’t be limited by these
#### a.	Do variables have missing/null values?


In [ ]:
df.isnull().sum()     

There are four missing values in Albumin_and_Globulin_Ratio.Filling missing values with mean.

In [ ]:
df['Albumin_and_Globulin_Ratio'].fillna(df['Albumin_and_Globulin_Ratio'].mean(), inplace=True)
df.info()

#### b.	Do variables have outliers? 

In [ ]:
cols = df.select_dtypes(exclude='object')
q=1
plt.figure(figsize=(15,20))
for col in cols:
   plt.subplot(5,5,q)
   ax = sns.boxplot(df[col],color='red')
   plt.xlabel(col)
   q+=1
plt.show()

There are outliers in our dataset but we are not removing off as of now.

#### c.	Is the Target distributed evenly? Is it a defect? If Yes, what steps are being taken to rectify the problem

In [ ]:
plt.subplot(1,2,1)
ax = sns.countplot(data=df, x='target')
plt.title('Liver patient', fontsize=15)
for i in ax.patches:
    
    ax.text(i.get_x()+0.3, i.get_height(), str(round(i.get_height(), 2)), fontsize=15, color='red')



As we can observe that data is not balanced so we will balance it by smote or nearmiss.

In [ ]:
print('We will be using smote when we will splitting the dataset in train and test')


### 4.Summarize relationships among variables (10 marks)               

#### a.	Plot relevant categorical plots. Find out which are the variables most correlated or appear to be in causation with Target? Do you want to exclude some variables from the model based on this analysis? What other actions will you take?

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(df.corr(), cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'coolwarm')
plt.title('Correlation between features');

#### The above correlation also indicates the following correlation
##### Total_Protiens & Albumin
##### Alamine_Aminotransferase & Aspartate_Aminotransferase
##### Direct_Bilirubin & Total_Bilirubin
##### There is some correlation between Albumin_and_Globulin_Ratio and Albumin. But its not as high as Total_Protiens & Albumin

In [ ]:
g = sns.FacetGrid(df, col="Gender", row="target", margin_titles=True)
g.map(plt.scatter,"Direct_Bilirubin", "Total_Bilirubin", edgecolor="w")
plt.subplots_adjust(top=0.9)

There seems to be direct relationship between Total_Bilirubin and Direct_Bilirubin. We have the possibility of removing one of this feature.

### b.	Plot all independent variables with the target & find out the relationship? Perform the Relevant Tests to find out if the Independent variables are associated with the Target Variable.

In [ ]:
# plotting all the dependent variables with target variable
# plotting scatter plot of continous variable with target variable
num_cols=['Age', 'Total_Bilirubin', 'Direct_Bilirubin',
     'Alkaline_Phosphotase', 'Alamine_Aminotransferase', 'Aspartate_Aminotransferase', 'Total_Protiens',
     'Albumin', 'Albumin_and_Globulin_Ratio']
q=1
plt.figure(figsize=(15,25))

for i in num_cols:
   plt.subplot(4,3,q)
   plt.title(i)
   plt.scatter(df[i],df["target"])
   plt.xlabel(i)
   plt.ylabel("target")
   q+=1
plt.show()

As we can see the relationship of target variables with each variables in the dataset.

In [ ]:
g = sns.FacetGrid(df, col="Gender", row="target", margin_titles=True)
g.map(plt.scatter,"Albumin_and_Globulin_Ratio", "Total_Protiens",  edgecolor="w")
plt.subplots_adjust(top=0.9)

In [ ]:
sns.jointplot("Albumin_and_Globulin_Ratio", "Albumin", data=df, kind="reg")

From the above jointplots and scatterplots, we find direct relationship between the following features:
Direct_Bilirubin & Total_Bilirubin
Aspartate_Aminotransferase & Alamine_Aminotransferase
Total_Protiens & Albumin
Albumin_and_Globulin_Ratio & Albumin

In [ ]:
# Converting gender into dummies
df = pd.get_dummies(df,columns = ['Gender'],drop_first = True)

In [ ]:
df.head()

### 5.Split dataset into train and test (70:30) 
#### a.	Are both train and test representative of the overall data? How would you ascertain this statistically?



In [ ]:
import scipy.stats as st

In [ ]:
X=df.drop('target', axis=1)
Y= df['target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train ,X_test, Y_train , Y_test = train_test_split(X , Y , test_size = 0.30 , random_state =42)
X_train.shape , Y_train.shape

H0:mu of Y_train=mu of Y_test =mu of Y
H1:any one of them differs

In [ ]:
st.f_oneway(Y_train,Y_test,Y)

P value is greater than 0.05 so we are fai to reject null hypothesis.so mean are same of all.

### Fit a base model and explain the reason of selecting that model. Please write your key observations.
#### a.	What is the overall Accuracy? Please comment on whether it is good or not. 
#### b.	What is Precision, Recall and F1 Score and what will be the optimization objective keeping in mind the problem statement.
#### c.	Which variables are significant?
#### d.	What is Cohen’s Kappa Value and what inference do you make from the model
#### e.	Which other key model output parameters do you want to look at? 



### Using base model Logistic Regression 

In [ ]:
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [ ]:
 ## using Logistic Regression
# Create logistic regression object
logreg = LogisticRegression()
# Train the model using the training sets and check score
logreg.fit(X_train, Y_train)
#Predict Output
log_predicted= logreg.predict(X_test)

logreg_score = round(logreg.score(X_train, Y_train) * 100, 2)
logreg_score_test = round(logreg.score(X_test, Y_test) * 100, 2)
#Equation coefficient and Intercept
print('Logistic Regression Training Score: \n', logreg_score)
print('Logistic Regression Test Score: \n', logreg_score_test)
print('Coefficient: \n', logreg.coef_)
print('Intercept: \n', logreg.intercept_)
print('Accuracy: \n', accuracy_score(Y_test,log_predicted))
print('Confusion Matrix: \n', confusion_matrix(Y_test,log_predicted))
print('Classification Report: \n', classification_report(Y_test,log_predicted))

sns.heatmap(confusion_matrix(Y_test,log_predicted),annot=True,fmt="d")

**Overall accuracy is the probability that an individual will be correctly classified by a test; that is, the sum of the true positives plus true negatives divided by the total number of individuals tested.**

**Precision - Precision is the ratio of correctly predicted positive observations to the total predicted positive observations. The question that this metric answer is of all passengers that labeled as survived, how many actually survived? High precision relates to the low false positive rate. We have got 0.788 precision which is pretty good.**

Precision = TP/TP+FP

**Recall (Sensitivity) - Recall is the ratio of correctly predicted positive observations to the all observations in actual class.**

Recall = TP/TP+FN

**F1 score - F1 Score is the weighted average of Precision and Recall.**
F1 Score = 2*(Recall * Precision) / (Recall + Precision)

In [ ]:
# reading the coefficient parameters
pd.DataFrame(zip(list(X_train.columns),list(logreg.coef_[0])), columns=["Variable","Coefficient"])\
.sort_values('Coefficient').style.background_gradient(cmap='viridis', low=0.2, high=0.2)


In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
print('Cohen Kappa Score on this model',cohen_kappa_score(Y_test,log_predicted))

Cohen's kappa coefficient is a statistic that is used to measure inter-rater reliability for qualitative items. It is generally thought to be a more robust measure than simple percent agreement calculation, as κ takes into account the possibility of the agreement occurring by chance. 

In [ ]:

print(f'Coefficients: {logreg.coef_}')
print(f'Intercept: {logreg.intercept_}')
print(f'R^2 score: {logreg.score(X, Y)}')

### 7. How do you improve the accuracy of the model? Write clearly the changes that you will make before re-fitting the model. Fit the final model. 
#### Please feel free to have any number of iterations to get to the final answer. Marks are awarded based on the quality of final model you are able to achieve. 



## Iteration 1

In [ ]:
# Hyper parameter tuning for logistic regression

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, binarize
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve


In [ ]:
# creating parameter grid
dict_params = {"penalty" : ["l1", "l2"],
               "C" : [0.001, 0.01, 0.1, 1, 10, 100]}

# hyperparameter tuning
base_model = LogisticRegression()
model_tuning = GridSearchCV(base_model, param_grid=dict_params, scoring="roc_auc", cv=4, return_train_score=True)
model_tuning.fit(X,Y)

# model results
cv_results = pd.DataFrame(model_tuning.cv_results_)
cv_results['train_test_diff'] = cv_results['mean_train_score'] - cv_results['mean_test_score']
cv_results.sort_values('train_test_diff')[["param_C","param_penalty","mean_train_score","mean_test_score",'train_test_diff']]

In [ ]:
# visualizing training and testing accuracy
plt.plot(cv_results.index+1, cv_results["mean_test_score"], label="test score")
plt.plot(cv_results.index+1, cv_results["mean_train_score"], label="train score")
plt.title("Training vs. Test score")
plt.ylabel("ROC AUC Score")
plt.xlabel("Iteration")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# fitting model with best params
model_logit = LogisticRegression(penalty='l1', C=100)
model_logit.fit(X_train, Y_train)
y_pred = model_logit.predict(X_test)

In [ ]:
# confusion matrix
cm = pd.DataFrame(confusion_matrix(Y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# plotting test results confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, cmap="RdYlGn", annot=True, cbar=False, square=True)
plt.xlabel("Predicted values", fontsize=12)
plt.ylabel("Actual values", fontsize=12)
plt.title("Test results", fontsize=20)
plt.show()

cm_reference = pd.DataFrame(np.array(["TN","FP","FN","TP"]).reshape(2,2), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])
print(cm_reference)

In [ ]:
# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)

In [ ]:
df_results = pd.DataFrame(columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])

In [ ]:
# itereation results
description = "Logit with hyperparamter tuning"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(Y_test,y_pred),2)
recall = round(recall_score(Y_test,y_pred),2)
accuracy = round(accuracy_score(Y_test,y_pred),2)
f1 = round(f1_score(Y_test,y_pred),2)
auc = round(roc_auc_score(Y_test,y_pred),2)

df_results = pd.concat([df_results,
                        pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])
                                  ], axis=0)

df_results

## Iteration 2

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# fitting base model with default params
model_knn = KNeighborsClassifier()
model_knn.fit(X_train, Y_train)

In [ ]:
# predicting X_test
y_pred = model_knn.predict(X_test)

# checking for model overfit
print("Training accuracy:", accuracy_score(Y_train,model_knn.predict(X_train)))
print("Test accuracy:", accuracy_score(Y_test,y_pred))

In [ ]:
# confusion matrix
cm = pd.DataFrame(confusion_matrix(Y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# plotting test results confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, cmap="RdYlGn", annot=True, cbar=False, square=True)
plt.xlabel("Predicted values", fontsize=12)
plt.ylabel("Actual values", fontsize=12)
plt.title("Test results", fontsize=20)
plt.show()

cm_reference = pd.DataFrame(np.array(["TN","FP","FN","TP"]).reshape(2,2), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])
print(cm_reference)

In [ ]:
# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)

In [ ]:
# itereation results
description = "Base kNN model"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(Y_test,y_pred),2)
recall = round(recall_score(Y_test,y_pred),2)
accuracy = round(accuracy_score(Y_test,y_pred),2)
f1 = round(f1_score(Y_test,y_pred),2)
auc = round(roc_auc_score(Y_test,y_pred),2)

df_results = pd.concat([df_results,
                        pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])
                                  ], axis=0)

df_results

In [ ]:
## Optimizing for optimal k value.

In [ ]:
# k values for model complexity
neig = np.arange(1, 25)
train_accuracy, test_accuracy = [], []

# loop over different values of k
for k in neig:
    # k from 1 to 25(exclude)
    knn = KNeighborsClassifier(n_neighbors=k)
    # Fit with knn
    knn.fit(X_train,Y_train)
    #train accuracy
    train_accuracy.append(knn.score(X_train, Y_train))
    # test accuracy
    test_accuracy.append(knn.score(X_test, Y_test))

# Plot
plt.figure(figsize=[12,8])
plt.plot(neig, test_accuracy, label = 'Testing Accuracy')
plt.plot(neig, train_accuracy, label = 'Training Accuracy')
plt.legend()
plt.title('Value VS Accuracy',fontsize=20)
plt.xlabel('Number of Neighbors',fontsize=20)
plt.ylabel('Accuracy',fontsize=20)
plt.xticks(neig)
plt.grid()
plt.show()
print("Best accuracy is {} with k = {}".format(np.max(test_accuracy),1+test_accuracy.index(np.max(test_accuracy))))

In [ ]:
# fitting base model with k=6
model_knn = KNeighborsClassifier(n_neighbors=6)
model_knn.fit(X_train, Y_train)

In [ ]:
# predicting X_test
y_pred = model_knn.predict(X_test)

# checking for model overfit
print("Training accuracy:", accuracy_score(Y_train,model_knn.predict(X_train)))
print("Test accuracy:", accuracy_score(Y_test,y_pred))

In [ ]:
# confusion matrix
cm = pd.DataFrame(confusion_matrix(Y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# plotting test results confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, cmap="RdYlGn", annot=True, cbar=False, square=True)
plt.xlabel("Predicted values", fontsize=12)
plt.ylabel("Actual values", fontsize=12)
plt.title("Test results", fontsize=20)
plt.show()

cm_reference = pd.DataFrame(np.array(["TN","FP","FN","TP"]).reshape(2,2), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])
print(cm_reference)

In [ ]:
# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)

In [ ]:
# itereation results
description = "kNN with optimal k (6)"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(Y_test,y_pred),2)
recall = round(recall_score(Y_test,y_pred),2)
accuracy = round(accuracy_score(Y_test,y_pred),2)
f1 = round(f1_score(Y_test,y_pred),2)
auc = round(roc_auc_score(Y_test,y_pred),2)

df_results = pd.concat([df_results,
                        pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])
                                  ], axis=0)

df_results

In [ ]:
# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
model_rfc = RandomForestClassifier()
model_rfc.fit(X_train, Y_train)

In [ ]:
# predicting X_test
y_pred = model_rfc.predict(X_test)

# checking for model overfit
print("Training accuracy:", accuracy_score(Y_train,model_knn.predict(X_train)))
print("Test accuracy:", accuracy_score(Y_test,y_pred))

In [ ]:
# confusion matrix
cm = pd.DataFrame(confusion_matrix(Y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# plotting test results confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, cmap="RdYlGn", annot=True, cbar=False, square=True)
plt.xlabel("Predicted values", fontsize=12)
plt.ylabel("Actual values", fontsize=12)
plt.title("Test results", fontsize=20)
plt.show()

cm_reference = pd.DataFrame(np.array(["TN","FP","FN","TP"]).reshape(2,2), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])
print(cm_reference)

In [ ]:
# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)

In [ ]:
# itereation results
description = "Base random forest"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(Y_test,y_pred),2)
recall = round(recall_score(Y_test,y_pred),2)
accuracy = round(accuracy_score(Y_test,y_pred),2)
f1 = round(f1_score(Y_test,y_pred),2)
auc = round(roc_auc_score(Y_test,y_pred),2)

df_results = pd.concat([df_results,
                        pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])
                                  ], axis=0)

df_results

**As we can see that logistic regression and KNN with k=6 is the best model for us as it gives best accuracy.**

### 8. Summarize as follows 
#### 1.	Summarize the overall fit of the model and list down the measures to prove that it is a good model
#### 2.	Write down a business interpretation/explanation of the model – which variables are affecting the target the most and explain the relationship. Feel free to use charts or graphs to explain.
#### 3.	What changes from the base model had the most effect on model performance?
#### 4.	What are the key risks to your results and interpretation?


1.

As we can see that KNN with k=6 is the best model for us as it gives best accuracy.We use optimum k value 6 by hyper parameter tuning.It gives 74% accuracy. Logistic regression is also giving almost same accuracy.

2.

In [ ]:
print('Correlation of selector with all independent variables')
df.corr()['target'].plot.barh()
plt.show()

Albumin and Albumin_and_Globulin ratio,Total_Protiens are the variables which are afftecting target variable most

3.

We are changing value of k by doing hyper parameter tuning. We get optimum k value 6 for the model.it increases the accuracy compared to base model.

4.

- Mitigating Risk of Machine Learning
- Data Difficulties.
- Technology Troubles.
- Security Snags.
- Models Misbehaving.
- Interaction Issues.